In [ ]:
%matplotlib inline
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
# Open SST and Wind together into a single xarray Dataset
mfdataset = xr.open_mfdataset(['../datasets/elnino/cci_sss_anomalies_2010_2019.nc', '../datasets/elnino/cci_sst_anomalies_1981_2018.nc','../datasets/elnino/ifremer_wind_anomalies_1992_2019.nc'])
mfdataset

In [ ]:
sst_anomaly = mfdataset['sst_anomaly']
wind_anomaly = mfdataset['eastward_wind_anomaly']
timedata = mfdataset['time']
wind_anomaly
timedata[126]

In [ ]:
# Restrict area to Nino 3.4
nino34 = sst_anomaly.sel(time=slice("1992-01-01","2018-12-01"), lat=slice(-5,5), lon=slice(190,240))
# wind34 = wind_anomaly.sel(time=slice("1992-01-01","2018-12-01"), lat=slice(-5,5), lon=slice(190,240))
wind34 = wind_anomaly.sel(time=slice("1992-01-01","2018-12-01"), lat=slice(-5,5), lon=slice(160,210))
time34 = timedata.sel(time=slice("1992-01-01","2018-12-01"))

# Compute Nino 3.4 index
nino34_timeseries = nino34.mean('lat').mean('lon')
nino34_index = nino34_timeseries.rolling(time=5).mean().dropna("time")
wind34_timeseries = wind34.mean('lat').mean('lon')
wind34_index = wind34_timeseries.rolling(time=5).mean().dropna("time")

# Plot Nino 3.4 index timeseries
fig = plt.figure(figsize=(8, 4))
ax = fig.gca()

nino34_index.plot(ax=ax)
# Plot threshold lines
start_time, end_time = nino34_index.get_index('time')[0], nino34_index.get_index('time')[-1]
plt.hlines(0.4, start_time, end_time, colors = 'black', linestyles = 'dashed')
plt.hlines(0, start_time, end_time, colors = 'black')
plt.hlines(-0.4, start_time, end_time, colors = 'black', linestyles = 'dashed')
plt.xlabel('date', fontsize=12)
plt.ylabel(b'Ni\xc3\xb1o-3.4 index'.decode("utf-8"), fontsize=12)

wind34_index.plot(ax=ax)
# Plot threshold lines
start_time, end_time = wind34_index.get_index('time')[0], wind34_index.get_index('time')[-1]
plt.hlines(0.4, start_time, end_time, colors = 'black', linestyles = 'dashed')
plt.hlines(0, start_time, end_time, colors = 'black')
plt.hlines(-0.4, start_time, end_time, colors = 'black', linestyles = 'dashed')
plt.xlabel('date', fontsize=12)
plt.ylabel(b'Ni\xc3\xb1o-3.4 index'.decode("utf-8"), fontsize=12)

plt.show()

In [ ]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
# Generate history dataset
tmin = 6
timelength = 12
timelength_ext = 24
tmax = tmin + timelength
tmax_ext = tmin + timelength_ext
nino34_hist = np.array([nino34_index.isel(time=slice(t0,320-tmax+t0)) for t0 in range(tmax-tmin)]).T
nino34_hist_ext = np.array([nino34_index.isel(time=slice(t0,320-tmax_ext+t0)) for t0 in range(tmax_ext-tmin)]).T
wind34_hist = np.array([wind34_index.isel(time=slice(t0,320-tmax+t0)) for t0 in range(tmax-tmin)]).T
total_hist = np.append(nino34_hist, wind34_hist, axis=1)
total_hist.shape

In [ ]:
X0 = nino34_hist
X1 = total_hist
X2 = nino34_hist_ext
B = nino34_index.isel(time=slice(tmax,320))
B_ext = nino34_index.isel(time=slice(tmax_ext,320))

train_size = 0.7
nsplit = int(round(total_hist.shape[0]*train_size))
train_X0, train_B = X0[:nsplit], B[:nsplit]
test_X0, test_B = X0[nsplit:], B[nsplit:]
train_X1, train_B = X1[:nsplit], B[:nsplit]
test_X1, test_B = X1[nsplit:], B[nsplit:]
train_X2, train_B2 = X2[:nsplit], B_ext[:nsplit]
test_X2, test_B2 = X2[nsplit:], B_ext[nsplit:]

In [ ]:
# Set up linear model
model_lr0 = LinearRegression()
# Fit linear model on training data
model_lr0.fit(train_X0, train_B)

# Set up linear model
model_lr1 = LinearRegression()
# Fit linear model on training data
model_lr1.fit(train_X1, train_B)

# Set up linear model
model_lr2 = LinearRegression()
# Fit linear model on training data
model_lr2.fit(train_X2, train_B2)

In [ ]:
def plot_subgrid(f, X, B, tmax, train_size=0.75, samplesize=4):
    """f is a function that takes an array of size (n_samples, n_features) and outputs
       an array of size (n_samples, n_features)
       X, B are arrays of size (n_samples, n_features)
    """
    Bpred = f(X)
#     print(Bpred.shape)
    tspan = np.linspace(0, tmax, X.shape[0])
    nsplit = int(round(nino34_hist.shape[0]*train_size))
    fig = plt.figure(figsize=(20, 6))
    ax = plt.subplot(2, 2, 1)
    plt.plot(time34[tmax+4:], Bpred, label='predicted')
    plt.plot(time34[tmax+4:], B, label='truth')
    Bmax = np.max(np.stack([Bpred, B]))
    Bmin = np.min(np.stack([Bpred, B]))
    plt.vlines(time34[nsplit+4], Bmin-1, Bmax+1, linestyles='dashed')
    ax.set_ylabel(fr'$B_{1}$', fontsize=15)
    ax.set_xlabel('time', fontsize=15)
    plt.show()
    
plot_subgrid(model_lr0.predict, X0, B, tmax, train_size)
plot_subgrid(model_lr1.predict, X1, B, tmax, train_size)
plot_subgrid(model_lr2.predict, X2, B_ext, tmax_ext, train_size)

In [ ]:
from sklearn.metrics import r2_score

Bpred0 = model_lr0.predict(test_X0)
R2_lr0 = r2_score(test_B, Bpred0)
print(f"Score for linear regression: {R2_lr0}")

Bpred1 = model_lr1.predict(test_X1)
R2_lr1 = r2_score(test_B, Bpred1)
print(f"Score for linear regression: {R2_lr1}")

Bpred2 = model_lr2.predict(test_X2)
R2_lr2 = r2_score(test_B2, Bpred2)
print(f"Score for linear regression: {R2_lr2}")

In [ ]:
from sklearn.linear_model import Lasso

a0 = a1 = a2 = 0.05

# Set up lasso model 
model_lasso0 = Lasso(alpha=a0)
# Fit lasso on training data
model_lasso0.fit(train_X0, train_B)

# Set up lasso model 
model_lasso1 = Lasso(alpha=a1)
# Fit lasso on training data
model_lasso1.fit(train_X1, train_B)

# Set up lasso model 
model_lasso2 = Lasso(alpha=a2)
# Fit lasso on training data
model_lasso2.fit(train_X2, train_B2)

In [ ]:
plt.bar(np.arange(1, timelength, 1), model_lasso0.coef_[0])
plt.hlines(0, xmin=0, xmax=timelength, linewidth=0.2)
plt.xlabel("components of X", fontsize=14)
plt.ylabel("coefficient", fontsize=14)
plt.title(r"coefficients use to predict $B_1$", fontsize=14)
plt.show()
print(f"Components of X with non-zero coefficients: {[i+1 for i in np.where(model_lasso0.coef_[0] != 0)[0]]}")

plt.bar(np.arange(1, timelength, 1), model_lasso1.coef_[0])
plt.hlines(0, xmin=0, xmax=timelength, linewidth=0.2)
plt.xlabel("components of X", fontsize=14)
plt.ylabel("coefficient", fontsize=14)
plt.title(r"coefficients use to predict $B_1$", fontsize=14)
plt.show()
print(f"Components of X with non-zero coefficients: {[i+1 for i in np.where(model_lasso1.coef_[0] != 0)[0]]}")

plt.bar(np.arange(1, timelength, 1), model_lasso2.coef_[0])
plt.hlines(0, xmin=0, xmax=timelength, linewidth=0.2)
plt.xlabel("components of X", fontsize=14)
plt.ylabel("coefficient", fontsize=14)
plt.title(r"coefficients use to predict $B_1$", fontsize=14)
plt.show()
print(f"Components of X with non-zero coefficients: {[i+1 for i in np.where(model_lasso2.coef_[0] != 0)[0]]}")

In [ ]:
plot_subgrid(model_lasso0.predict, X0, B, tmax)
plot_subgrid(model_lasso1.predict, X1, B, tmax)
plot_subgrid(model_lasso2.predict, X2, B_ext, tmax_ext)

In [ ]:
Bpred0 = model_lasso0.predict(test_X0)
R2_lasso0 = r2_score(test_B, Bpred0)
print(f"Score for lasso: {R2_lasso0}")

Bpred1 = model_lasso1.predict(test_X1)
R2_lasso1 = r2_score(test_B, Bpred1)
print(f"Score for lasso: {R2_lasso1}")

Bpred2 = model_lasso2.predict(test_X2)
R2_lasso2 = r2_score(test_B2, Bpred2)
print(f"Score for lasso: {R2_lasso2}")

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Set up decision tree
model_dt0 = DecisionTreeRegressor()
# Fit decision tree
model_dt0.fit(train_X0, train_B)

# May take 20 ~ 30 seconds to compute
model_rf0 = RandomForestRegressor(max_depth=10)
model_rf0.fit(train_X0, train_B)

# Set up decision tree
model_dt1 = DecisionTreeRegressor()
# Fit decision tree
model_dt1.fit(train_X1, train_B)

# May take 20 ~ 30 seconds to compute
model_rf1 = RandomForestRegressor(max_depth=10)
model_rf1.fit(train_X1, train_B)

# Set up decision tree
model_dt2 = DecisionTreeRegressor()
# Fit decision tree
model_dt2.fit(train_X2, train_B2)

# May take 20 ~ 30 seconds to compute
model_rf2 = RandomForestRegressor(max_depth=10)
model_rf2.fit(train_X2, train_B2)

In [ ]:
# Plot predictions
plot_subgrid(model_dt0.predict, X0, B, tmax)
plot_subgrid(model_rf0.predict, X0, B, tmax)
plot_subgrid(model_dt1.predict, X1, B, tmax)
plot_subgrid(model_rf1.predict, X1, B, tmax)
plot_subgrid(model_dt2.predict, X2, B_ext, tmax_ext)
plot_subgrid(model_rf2.predict, X2, B_ext, tmax_ext)

In [ ]:
Bpred0 = model_dt0.predict(test_X0)
R2_dt0 = r2_score(test_B, Bpred0)
print(f"Score for decision tree: {R2_dt0}")

Bpred0 = model_rf0.predict(test_X0)
R2_rf0 = r2_score(test_B, Bpred0)
print(f"Score for lasso: {R2_rf0}")

Bpred1 = model_dt1.predict(test_X1)
R2_dt1 = r2_score(test_B, Bpred1)
print(f"Score for decision tree: {R2_dt1}")

Bpred1 = model_rf1.predict(test_X1)
R2_rf1 = r2_score(test_B, Bpred1)
print(f"Score for lasso: {R2_rf1}")

Bpred2 = model_dt1.predict(test_X2)
R2_dt2 = r2_score(test_B2, Bpred2)
print(f"Score for decision tree: {R2_dt2}")

Bpred2 = model_rf1.predict(test_X2)
R2_rf2 = r2_score(test_B2, Bpred2)
print(f"Score for lasso: {R2_rf2}")